In [1]:
# Task: Extract motion features based on raw pixel values using FFT
# Input: Videos after temporal segmentation (detect the portion of the video where task 2 is performed)
# Input Directory: C:\Wasif\PD Motor Feature Extraction\TemporalSegmentOutput
# Output: FFT features based on raw pixel values
# Output Directory: C:\Wasif\PD Motor Feature Extraction\RawPixelFeatures

#First, process a single video
# video -> 
# Nf = number of frames in the video
# Np = Number of pixels in the video = (height x width)
# First convert to a Np x Nf array (Pxy = Nf length vector for each pixel)
# Run FFT on sequence Pxy with sampling rate = 128
# Each Pxy vector will be encoded to a 128 length vector of frequency response
# Output is now Np x 128 matrix

import mediapipe as mp
import numpy as np
import cv2
import os
import PIL
import pickle

#print (cv2.__version__)

INPUT_DIR = "C:\\Wasif\\PD Motor Feature Extraction\\TemporalSegmentOutput\\"
OUTPUT_DIR = "C:\\Wasif\\PD Motor Feature Extraction\\RawPixelFeatures\\"
OUTPUT_PLOT_DIR = "C:\\Wasif\\PD Motor Feature Extraction\\RawPixelFeatures\\Plots\\"

FOURIER_SAMPLES = 128
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
MVMNT_THRESHOLD = 0.2

assert(os.path.exists(INPUT_DIR))

if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)
    
if not os.path.exists(OUTPUT_PLOT_DIR):
    os.mkdir(OUTPUT_PLOT_DIR)

In [2]:
#List the files
file_list = [filename for filename in os.listdir(INPUT_DIR) if '.mp4' in filename]
file_list_fullpath = [os.path.join(INPUT_DIR,filename) for filename in file_list]

print(file_list_fullpath)

['C:\\Wasif\\PD Motor Feature Extraction\\TemporalSegmentOutput\\2017-08-18T14-59-52-530Z49-task2-temp.mp4', 'C:\\Wasif\\PD Motor Feature Extraction\\TemporalSegmentOutput\\2017-08-18T15-24-14-004Z53-task2-temp.mp4', 'C:\\Wasif\\PD Motor Feature Extraction\\TemporalSegmentOutput\\2017-08-22T02-01-21-948Z87-task2-temp.mp4', 'C:\\Wasif\\PD Motor Feature Extraction\\TemporalSegmentOutput\\2017-09-22T18-38-44-872Z33-task2-temp.mp4', 'C:\\Wasif\\PD Motor Feature Extraction\\TemporalSegmentOutput\\2017-09-28T14-17-07-280Z18-task2-temp.mp4', 'C:\\Wasif\\PD Motor Feature Extraction\\TemporalSegmentOutput\\2017-10-04T14-04-32-851Z2-task2-temp.mp4', 'C:\\Wasif\\PD Motor Feature Extraction\\TemporalSegmentOutput\\2017-10-10T21-15-21-095Z89-task2-temp.mp4', 'C:\\Wasif\\PD Motor Feature Extraction\\TemporalSegmentOutput\\2017-10-12T20-07-10-147Z53-task2-temp.mp4', 'C:\\Wasif\\PD Motor Feature Extraction\\TemporalSegmentOutput\\2017-10-13T05-38-01-106Z41-task2-temp.mp4', 'C:\\Wasif\\PD Motor Feature

In [3]:
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    num_frames = (int)(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    pixel_values = np.zeros((IMAGE_HEIGHT, IMAGE_WIDTH, num_frames))
    fft_values = np.zeros((IMAGE_HEIGHT, IMAGE_WIDTH, FOURIER_SAMPLES))
    #print(num_frames)
    current_frame_id = 0
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            #print("Ignoring empty camera frame.")
            break

        
        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
            
        # Print nose landmark.
        image_height, image_width, _ = image.shape
        assert(image_height==IMAGE_HEIGHT and image_width==IMAGE_WIDTH)
        
        #copy all the pixels of the current frame, take mean to consider gray-scale
        pixel_values[:,:,current_frame_id] = np.mean(image, axis = 2)
        
        #print("test")
        #print(image[20,20])
        #print(np.mean(image, axis=2)[20][20])
        #print(pixel_values[20,20,current_frame_id])
        
        #for x in range(0, image_hight):
        #    for y in range(0, image_width):
        #        pixel_values[x,y,current_frame_id] = np.mean(image[x,y])
        
        #print(image_hight, image_width)
        current_frame_id +=1
        if cv2.waitKey(5) & 0xFF == 27:
            break
        
    cap.release()
    cv2.destroyAllWindows()
    
    fft_values= np.fft.fft(pixel_values, n=FOURIER_SAMPLES, axis=2).real
    
    #for x in range(0, image_height):
    #    for y in range(0, image_width):
            #print(pixel_values[x,y].shape)
            #Run FFT on pixel_values[x,y]
            #A = np.fft.fft(pixel_values[x,y,:], n=FOURIER_SAMPLES)
            #print(A.shape)
     #       fft_values[x,y,:] = np.fft.fft(pixel_values[x,y,:], n=FOURIER_SAMPLES).real
            #print(fft_values[x,y].shape)
    
    return fft_values

In [4]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from PIL import Image, ImageFilter 

X = {}

i = 0
for file_path in file_list_fullpath:
    file_id = file_path[file_path.rfind("\\")+1:]
    file_id = file_id[0:file_id.find("-temp.mp4")]
    
    X[file_id] = {}
    
    i+=1
    #if i>=5:
    #    break
    
    if(i%20==0):
        print(i)
    
    #process_one_video_hand(file_path)
    try:
        fft_values = process_video(file_path)
        
        A = np.zeros((IMAGE_HEIGHT, IMAGE_WIDTH))
        
        for x in range(0, IMAGE_HEIGHT):
            for y in range(0, IMAGE_WIDTH):
                A[x,y] = np.dot(np.abs(np.fft.fftfreq(FOURIER_SAMPLES)), fft_values[x,y,:])
        
        
        pixel_movements = A.flatten() #256x256
        frequency_components = np.mean(fft_values,axis=(0,1)).flatten() #128
        
        assert(len(pixel_movements)==IMAGE_HEIGHT*IMAGE_WIDTH and len(frequency_components)==FOURIER_SAMPLES)
        
        X[file_id]["pixel_movements"] = pixel_movements
        X[file_id]["frequency_components"] = frequency_components
        
        #fig, ax = plt.subplots()
        #im = ax.imshow(A, cmap='gray')
        #plt.show()
        plot_file_name = file_path[file_path.rfind("\\")+1:]
        plot_file_name = plot_file_name[0:plot_file_name.find("-temp.mp4")]
        #plt.savefig(OUTPUT_PLOT_DIR+plot_file_name)
        #plt.show()
        #plt.close()
        
        A = (((A - A.min()) / (A.max() - A.min())) * 255.9).astype(np.uint8)
        im = Image.fromarray(A)
        #im = im.filter(ImageFilter.MedianFilter(size = 3))
        #im.show()
        im.save(OUTPUT_PLOT_DIR+plot_file_name+".bmp")
        
        plot_file_name +="_median3"
        im = im.filter(ImageFilter.MedianFilter(size = 3))
        im.save(OUTPUT_PLOT_DIR+plot_file_name+".bmp")

    except Exception as e:
        print("Error processing file: ")
        print(file_path)
        print(e)
        
pickle_filename = OUTPUT_DIR+"raw_pixel_features.pickle"
with open(pickle_filename, 'wb') as handle:
    pickle.dump(X, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(pickle_filename, 'rb') as handle:
    X_reloaded = pickle.load(handle)  

20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180


In [5]:
file_path = 'C:\\Wasif\\PD Motor Feature Extraction\\TemporalSegmentOutput\\2018-09-02T15-29-09-517Z80-task2-temp.mp4'
file_id = file_path[file_path.rfind("\\")+1:]
file_id = file_id[0:file_id.find("-temp.mp4")]
print(file_id)

2018-09-02T15-29-09-517Z80-task2
